This notebook is for transforming the full format form (xml) into the required formate for the input layer (1x455)


In [70]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
from datetime import datetime

In [15]:
file = 'test-race-form.xml'
result_master = pd.read_csv('masterfile.csv')

In [16]:
result_master.head()

,Unnamed: 0,race_id,track,dist,grade,place,dog_id,box,split_times,split_margins,run_time
0,0,208756700,Ballarat,450m,Maiden Heat,1.0,1296100098,2,6.73,0.00,26.17
1,1,208756700,Ballarat,450m,Maiden Heat,2.0,1177320005,3,6.87,0.14,26.18
2,2,208756700,Ballarat,450m,Maiden Heat,3.0,1549980026,5,6.99,0.26,26.26
3,3,208756700,Ballarat,450m,Maiden Heat,4.0,1474670055,7,6.88,0.15,26.36
4,4,208756700,Ballarat,450m,Maiden Heat,5.0,913850094,8,6.91,0.18,26.42


In [109]:
root = ET.parse(file).getroot()

track = root[0]
print("track =", track.text)
date = datetime.strptime(root[1].text, "%d %b %y")
print("date =", date)

for race in root.iter('Race'):
    print("race id = ",race.attrib["id"])
    print("Distance =", race[4].text, "\n")
    print("Grade =", race[5].text, "\n")

    print("box", "dog id", "DID IT RUN?")
    for dog in race.iter('Dog'):
        box = dog[0].text #1
        dog_id = dog.attrib["id"]
        besttime = dog[2].text #2
        #need to edit both of these to get the numbers out
        TOT, TOT1, TOT2, TOT3 = basic_form(dog[4].text)
        TTD, TTD1, TTD2, TTD3 = basic_form(dog[5].text)
        
        dogprize = dog.find('DogPRIZE').text
        run = diditrun(result_master,  race.attrib["id"], dog_id)
        forms = dog.findall('Form')
        print(box, dog_id, besttime, TOT, TTD, dogprize, run)
        print("\n----------\n Form\n place box dist age margin dog_time win_time split")
        for form in forms:
            if not form:
                break
            place = form[0].text[0]
            form_box = form[1].text
            form_dist = form[4].text
            form_date = datetime.strptime(form[5].text, "%d%b%y")
            form_age = (date - form_date).days
            #same track ???
            
            if place == "F":
                margin = 0
            else: 
                margin = form[8].text[:-1]
            dog_time = form[10].text
            win_time = form[11].text
            split = form[15].text
            print(place, form_box, form_dist, form_age, margin, dog_time, win_time, split)
            
            
            
            
        print("\n----------\n")
    break



track = Ballarat
date = 2017-04-01 00:00:00
race id =  208756700
Distance = 450m 

Grade = Maiden Heat 

box dog id DID IT RUN?
1 1096450034 FSH 5 0 140 yes

----------
 Form
 place box dist age margin dog_time win_time split
6 2 390 58 12.00 23.29 22.48 9.19
4 2 390 51 8.00 22.87 22.33 8.81
3 2 450 44 4.00 25.98 25.71 6.67
7 2 450 37 10.00 26.40 25.73 6.71
7 6 390 25 16.00 23.68 22.59 9.26

----------

2 1296100098 NBT 20 1 1125 yes

----------
 Form
 place box dist age margin dog_time win_time split
4 3 400 29 3.5 23.27 23.04 8.55
2 7 400 18 1.75 23.20 23.09 8.55
5 4 400 15 16.00 23.91 22.82 8.93
4 3 460 11 5.00 26.87 26.52 6.76
2 8 350 6 3.25 19.76 19.53 6.70

----------

3 1177320005 NBT 9 1 445 yes

----------
 Form
 place box dist age margin dog_time win_time split
2 1 350 71 1.00 19.96 19.91 6.71
3 6 350 62 1.75 19.66 19.54 6.80
3 5 450 58 2.25 25.87 25.72 6.72
4 6 460 46 7.50 26.82 26.32 6.85
4 5 450 16 1.5 26.15 26.04 6.79

----------

4 1805750008 FSH 0 0 0 yes

----------
 F

In [88]:
def diditrun(df, race_id, dog_id):
    #print(race_id)
    #print(dog_id)
    if not dog_id:
        return "no"
    
    result = df[(df["race_id"]==int(race_id)) & (df["dog_id"]==int(dog_id))]
    if result.empty:
        return "no"
    else:
        return "yes"

In [102]:
def basic_form(form):
    split_1 = form.split(" ")
    split_2 = split_1[1].split("-")
    return split_2

In [29]:
df = result_master
print(df[(df['race_id'] == 208756700) & (df["dog_id"]== 1096450034)])

   Unnamed: 0    race_id     track  dist        grade  place      dog_id  box  \
6           6  208756700  Ballarat  450m  Maiden Heat    7.0  1096450034    1   

   split_times  split_margins  run_time  
6         6.92           0.19     26.87  


In [74]:
dt = datetime.strptime("09Feb17", "%d%b%y")
dt

datetime.datetime(2017, 2, 9, 0, 0)

In [103]:
basic_form("Starts 5-0-0-1")

['5', '0', '0', '1']